In [1]:
import pandas as pd
from statsbombpy import sb
from mplsoccer import Pitch
import json
import requests
import ast
import matplotlib.pyplot as plt
import time
from kloppy import statsperform
from io import BytesIO
import matplotlib.patches as patches
from matplotlib.patches import Polygon
from kloppy import wyscout
import numpy as np
import glob
import time

Statsbomb

WYScout

xG, xT Calc.

clubelo